## Starfield

Hagamos primero los _imports_ necesarios:

In [1]:
import pygame
import random
import time

Y definamos algunas de las constantes que vamos a usar: El tamaño de la
pantalla y algunos colores básicos:

In [41]:
# Tamaño de pantalla

SIZE = WIDTH, HEIGHT = 800, 640

# Colores
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (0, 255, 255)

# Velocidad del juego
FPS = 24


### La clase Star

Partamos de la defición de la clase Star que hicimos el otro día. Solo tiene
una pequeña modificación, no se le pasa ningún parámetro (Antes se le pasaba 
las coordenadas x e y, ahora se calcula una posición al azar en el procedimiento
especial de inicialización, `__init__`):

In [11]:
class Star:
    
    def __init__(self):
        self.x = random.randrange(0, WIDTH)
        self.y = random.randrange(0, HEIGHT)
        
    def left(self):
        self.x = self.x - 1
        
    def right(self):
        self.x = self.x + 1

    def up(self):
        self.y = self.y - 1
        
    def down(self):
        self.y = self.y + 1


### El bucle principal del juego

El cuerpo principal de la animación, como lo dejamos el último día, con las estrellas moviendose hacia
la izquierda, y con una pequeña modificación:

In [12]:
pygame.init()
try:
    pygame.display.set_caption("Starfield")
    screen = pygame.display.set_mode(SIZE, 0, 24)

    # Parte de inicialización del juego
    stars = [Star() for _ in range(50)]
    clock = pygame.time.Clock()    
    in_game = True  # Indicador lógico para saber cuando debemos terminar el juego

    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        for star in stars:
            star.left()
        # Representamos el nuevo estado
        screen.fill(BLACK)
        for star in stars:
            pygame.draw.circle(screen, WHITE, (star.x, star.y), 2)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

Como vemos, casi todo el código está incluida ahora en un bloque de
control que no habiamos usado hasta ahora; se trata de una sentencia `try` (intentar en Inglés).

Usando esta sentencia podemos tratar determinados casos de error. En nuestro
caso, usamos una versión muy sencilla del `try`, la combinacion `try` y `finally`.

Esta combinación una nos permite definir que, si se produce
cualquier tipo de error, se ejecute siempre el bloque que viene a continuacion de la
sentencia `finally`. En este caso, si se produce _cualquier tipo de error_ dentro del
código del `try`, Python nos garantiza que ejecutará el codigo definido en la sentencia
`finally`, que en este caso es simplemente llamar a `pygame.quit()`.

De esta forma, la ventana del juego se terminará cerrando siempre, ya sea porque
hemos pulsado el botón de salir de la ventana o porque se haya producido cualquier
tipo de error en el código dentro del `try`.

### Lo que vamos a hacer hoy

1) Vamos a cambiar la definicion de la clase estrella (`Star`) para 
cambiar determinados parámetros: Color, tamaño, velocidad...

2) Vamos a introducir un poco de control de ventos, de forma que podamos, con una
techa, cambiar el movimiento de las estrellas al que queramos, arriba, abajo, derecha, izquierda.

3) Vamos a evitar que las estrellas desaparezcan de la pantalla, si una estrella desaparece por un lado
de la ventana, debe aparecer por el otro

**Ejercicio: Incluir un campo color en la clase `Star`**, de forma que cada estrella tenga un color
diferente. Recuerda que los colores son tuplas de tres elementos, con los
componentes rojo, verde y azul. Hay que tocar ahora mismo en dos partes del
código, en el inicializador (método *mágico* `__init__`) para crear el atributo
y en el bucle de pintado de las estrellas, para que tenga en cuenta este nuevo dato.

Modificar la siguiente celda:

In [5]:
class Star:
    
    def __init__(self):
        self.x = random.randrange(0, WIDTH)
        self.y = random.randrange(0, HEIGHT)
        ## define el color, al azar, aquí
        
    def left(self):
        self.x = self.x - 1
        
    def right(self):
        self.x = self.x + 1

    def up(self):
        self.y = self.y - 1
        
    def down(self):
        self.y = self.y + 1


Y prueba ejecutando de nuevo el bucle del programa. Aquí tendras que cambiar una línea
para que el programa use el nuevo atributo `color`:

In [6]:
pygame.init()
try:
    pygame.display.set_caption("Starfield")
    screen = pygame.display.set_mode(SIZE, 0, 24)

    # Parte de inicialización del juego
    stars = [Star() for _ in range(50)]
    clock = pygame.time.Clock()    
    in_game = True  # Indicador lógico para saber cuando debemos terminar el juego

    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        for star in stars:
            star.left()
        # Representamos el nuevo estado
        screen.fill(BLACK)
        for star in stars:
            pygame.draw.circle(screen, WHITE, (star.x, star.y), 2) # Esta es la línea a cambiar
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

### Refactorización

Según la wikipedia, la refectorizacion de código se define asi:

> **Refactorización de código**
> En ingeniería del software, el término refactorización se usa a menudo para describir la modificación del código fuente sin cambiar su comportamiento, lo que se conoce informalmente por limpiar el código. La refactorización se realiza a menudo como parte del proceso de desarrollo del software: los desarrolladores alternan la inserción de nuevas funcionalidades y casos de prueba con la refactorización del código para mejorar su consistencia interna y su claridad. Los tests aseguran que la refactorización no cambia el comportamiento del código.

Vamos a añadir dos métodos a la clase `Star` que nos facilitaran las modificaciones que vamos a realizar. 
En primer lugar crearemos un método `update`, que sera llamada por el bucle del juego para que la estrella
actualice su estado. En segundo lugar, crearemos un método `draw`, al que le pasaremos la superficie donde queremos
que se pinte la estrella, y asi será la propia estrella la que define como se pintará en la pantalla. De esta forma, podemos tener
un bucle principal generico, que funcionara para cualquier objeto que tenga un método `update` y `draw` como los descritos.

In [7]:
class Star:
    
    def __init__(self):
        self.x = random.randrange(0, WIDTH)
        self.y = random.randrange(0, HEIGHT)
        self.color = (
            random.randrange(128, 255),
            random.randrange(128, 255),
            random.randrange(128, 255),
            )
        
    def left(self):
        self.x = self.x - 1
        
    def right(self):
        self.x = self.x + 1

    def up(self):
        self.y = self.y - 1
        
    def down(self):
        self.y = self.y + 1
        
    def update(self):
        self.left()
        
    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (self.x, self.y), 2)
        


Y ahora podemos definir el bucle principal del juego para que functione con esta clase nueva de
estrellas. Además lo haremos como una función para poder llamar al juego cuando sea necesario sin tener
que repetir la celda cada vez:

In [8]:
def play_game():
    pygame.init()
    try:
        pygame.display.set_caption("Starfield 2")
        screen = pygame.display.set_mode(SIZE, 0, 24)
        # Parte de inicialización del juego
        stars = [Star() for _ in range(50)]
        clock = pygame.time.Clock()    
        in_game = True  # Indicador lógico para saber cuando debemos terminar el juego

        while in_game:
        # Obtener datos de entrada
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    in_game = False
            # Recalcular el estado del juego, en base al estado actual y a las entradas
            for star in stars:
                star.update()  # pasamos la responsabilidad de actualizar su estado a la estrella
            # Representamos el nuevo estado
            screen.fill(BLACK)
            for star in stars:
                star.draw(screen)  # pasamos la responsabilidad de dibujarse a la estrella
            pygame.display.update()
            clock.tick(FPS)
    finally:
        pygame.quit()

Para llamar a la función, simplemente la llamamos, con `play_game()`:

In [9]:
play_game()

Algunas cosas a tener en cuenta:
    
1) El programa sigue funcionando exactamente igual. Es importantante cuando refactorizemos centrarnos
  solo en reescribir el código para que sea más optimo o más legible, pero que se comporte
  en los demás aspectos igual que antes.

2) El programa funciona porque la función `play_game` y los objetos creados a partir
  de la clase `Star` tienen un acuerdo o contrato, en este caso que las estrellas
  se crean sin necesidad de parámetros, y que cada objeto de tipo estrella tiene unos
  métodos llamados `update` y `draw`, definidos de una forma precisa.

### Ejercicio: Añadir diferentes velocidades a las estrellas

> Cambiar la clase Star para añadir un atributo `speed` (Velocidad). De nuevo 
podemos calcular una velocidad al azar entre 1 y 5 pixels por frame. Habrá que modificar
los métodos `left`, `right`, `up` y `down` para que hagan su trabajo teniendo en cuenta
este nuevo atributo

In [65]:
class Star:
    
    def __init__(self):
        self.x = random.randrange(0, WIDTH)
        self.y = random.randrange(0, HEIGHT)
        self.color = (
            random.randrange(128, 255),
            random.randrange(128, 255),
            random.randrange(128, 255),
            )
        self.speed = ...  # Modificar esta línea para calcular una velocidad en el rango 1..5
        
    def left(self):
        self.x = self.x - 1  # También habra que tocar este métodos y usar el nuevo atributo
        
    def right(self):
        self.x = self.x + 1  # ...y este

    def up(self):
        self.y = self.y - 1  # ...y este
        
    def down(self):
        self.y = self.y + 1  # ...y también este
        
    def update(self):
        self.left()
        
    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (self.x, self.y), 2)
        


Ejecutemos el juego con la nueva definicón de `Star`:

In [66]:
play_game()

### Ejercicio para casa

Añadir un atributo `size` (tamaño en inglés), que sea un número entero entre 1 y 3, de forma
que podamos tener estrellas de diferentes tamaños. Pista: hay que modificar el código
del método inicializador `__init__` y también el mñetodo `draw`.
    

### Ejercicio, tratar eventos del teclado para controlar el movimiento de las estrellas

Primero, veamos un poco más de los enventos, que ahora mismo solo estamnos tratando los
eventos de salida. Vamos a hacer un programa pygame que nos muestre todos los eventos
que llegan en realidad:

In [4]:
pygame.init()
try:
    pygame.display.set_caption("Eventos")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    # Parte de inicialización del juego
    font = pygame.font.Font('./Hack-Regular.ttf', 18)
    lista_eventos = []
    clock = pygame.time.Clock()    
    in_game = True  # Indicador lógico para saber cuando debemos terminar el juego
    while in_game:
    # Obtener datos de entrada
        for event in pygame.event.get():
            lista_eventos.append(str(event))
            if event.type == pygame.QUIT:
                in_game = False
        if len(lista_eventos) > 35:
            lista_eventos = lista_eventos[-35:]
        screen.fill(BLACK)
        line = 0
        for texto in lista_eventos:
            s = font.render(texto, True, GREEN, BLACK)
            pos = (12, line*20)
            screen.blit(s, pos)
            line += 1
        pygame.display.update()
        clock.tick(FPS)
except Exception as err:
    print(err)
finally:
    print('Salimos')
    pygame.quit()

Salimos


He añadido una cláusula except. Esta se ejecutará cuando se detecte un tipo de
excepción. Si usamos `Exception`, le indicamos que cualquier error (Una práctica nada
recomendable).

Los eventos que nos llegan al programa son objetos de tipo evento, pero
hay varias clases, cada una con sus atributos y métodos correspondientes.
Por ejemplo, los eventos de tipo movimiento del raton incluyen, entre
otros datos, las coordenadas x y y del raton. Los eventos del tipo teclado, sin
embargo, incluyen el caracter que hemos pulsado en el teclado.

Lo único que tienen en comun todos los eventos es el atributo `type`. En el
código actual solo miramos que haya llegado un evento de tipo `pygame.QUIT`, que nos
indica que el sistema operativo quiere cerrar el programa. Pero hay muchos tipos
de eventos:
    
- QUIT
- ACTIVEEVENT
- KEYDOWN
- KEYUP
- MOUSEMOTION
- MOUSEBUTTONUP
- MOUSEBUTTONDOWN
- JOYAXISMOTION
- JOYBALLMOTION
- JOYHATMOTION
- JOYBUTTONUP
- JOYBUTTONDOWN
- VIDEORESIZE
- VIDEOEXPOSE
- USEREVENT

De todos estos, nos interesan especialmente los de teclado (`KEYDOWN`, `KEYUP`) y los del ratón (`MOUSEMOTION`, `MOUSEBUTTONUP` y `MOUSEBUTTONDOWN`). Para los eventos
de teclado y los que están asociados a los botones del ratón, vemos que hay dos eventos
en realidad, uno cuando se pulsa el boton y otro cuando la tecla deja de ser pulsada.

Pos ahora, nos interesan los eventos del teclado. Vamos a intentar que el campo
de estrellas se pueda mover usando las teclas `A`, `W`, `S` y `D`:

Como primer paso, vamos simplemente a detectar cada una de estas cuatro teclas, filtrando
los eventos que nos interesen:

In [1]:
import pygame
import random
import time

In [2]:
# Tamaño de pantalla

SIZE = WIDTH, HEIGHT = 800, 640

# Colores
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (0, 255, 255)

# Velocidad del juego
FPS = 30


In [8]:
pygame.init()
try:
    pygame.display.set_caption("Eventos")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    # Parte de inicialización del juego
    clock = pygame.time.Clock()    
    font = pygame.font.Font('./Hack-Regular.ttf', 18)
    lista_eventos =  []
    in_game = True  # Indicador lógico para saber cuando debemos terminar el juego
    while in_game:
    # Obtener datos de entrada
        for event in pygame.event.get():
            
            if event.type == pygame.QUIT:
                in_game = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    in_game = False
                elif event.key in (pygame.K_a, pygame.K_w, pygame.K_s, pygame.K_d):
                    print('Pulsada una de las teclas que nos interesa')
                    print(event)
                    lista_eventos.append(str(event))
                
        if len(lista_eventos) > 35:
            lista_eventos = lista_eventos[-35:]
        screen.fill(BLACK)
        line = 0
        for texto in lista_eventos:
            s = font.render(texto, True, GREEN, BLACK)
            pos = (12, line*20)
            screen.blit(s, pos)
            line += 1
        pygame.display.update()
        clock.tick(FPS)
except Exception as err:
    print(err)
finally:
    print('Salimos')
    pygame.quit()
    print('OK')

Pulsada una de las teclas que nos interesa
<Event(2-KeyDown {'key': 119, 'scancode': 25, 'mod': 0, 'unicode': 'w'})>
Pulsada una de las teclas que nos interesa
<Event(2-KeyDown {'key': 100, 'scancode': 40, 'mod': 0, 'unicode': 'd'})>
Pulsada una de las teclas que nos interesa
<Event(2-KeyDown {'key': 115, 'scancode': 39, 'mod': 0, 'unicode': 's'})>
Pulsada una de las teclas que nos interesa
<Event(2-KeyDown {'key': 97, 'scancode': 38, 'mod': 0, 'unicode': 'a'})>
Salimos
OK


Ahora, lo que nos interesa es que, en el método update de la estrella, sepa
en que direccion debe moverse. Para eso, vamos a crear una variable global con
5 posibles valores, que serian: Parado, hacia la izquierda, hacia la derecha, 
hacia arriba y hacia abajo.

In [40]:
STOP = 0
LEFT = 1
RIGHT = 2
UP = 3
DOWN = 4
STATUS = STOP

Redefinimos `Star` para que cuando se actualize cada estrella (método `update`)
tenga en cuenta el sentido del movimiento:

In [45]:
class Star:
    
    def __init__(self):
        self.x = random.randrange(0, WIDTH)
        self.y = random.randrange(0, HEIGHT)
        self.color = (
            random.randrange(128, 255),
            random.randrange(128, 255),
            random.randrange(128, 255),
            )
        self.speed = random.randrange(1, 5)
        self.size = random.randrange(1, 3)
        
    def left(self):
        self.x = self.x - self.speed
        
    def right(self):
        self.x = self.x + self.speed

    def up(self):
        self.y = self.y - self.speed
        
    def down(self):
        self.y = self.y + self.speed
        
    def update(self):
        global STATUS
        if STATUS == LEFT:
            self.left()
        elif STATUS == RIGHT:
            self.right()
        elif STATUS == UP:
            self.up()
        elif STATUS == DOWN:
            self.down()
        
    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (self.x, self.y), self.size)

Ahora ponemos la variable STATUS al valor correspondiente para cada tecla:

In [49]:
pygame.init()
try:
    pygame.display.set_caption("Eventos")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    # Parte de inicialización del juego
    STATUS = STOP
    clock = pygame.time.Clock()    
    stars =  [Star() for _ in range(75)]
    in_game = True  # Indicador lógico para saber cuando debemos terminar el juego
    while in_game:
    # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    in_game = False
                elif event.key == pygame.K_a:
                    STATUS = LEFT
                elif event.key == pygame.K_d:
                    STATUS = RIGHT
                elif event.key == pygame.K_w:
                    STATUS = UP
                elif event.key == pygame.K_s:
                    STATUS = DOWN
                elif event.key == pygame.K_SPACE:
                    STATUS = STOP
        for star in stars:
            star.update()
        screen.fill(BLACK)
        for star in stars:
            star.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
except Exception as err:
    print(err)
finally:
    print('Salimos')
    pygame.quit()
    print('OK')

Salimos
OK


In [47]:
pygame.K_SPACE

AttributeError: module 'pygame' has no attribute 'K_Space'